In [9]:
print(__doc__)

import random
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
import os
import csv
import pprint
import time

pp = pprint.PrettyPrinter(indent=4)

Automatically created module for IPython interactive environment


In [5]:
# Read training dataset
dataset_train = []
reader = csv.DictReader(open('fashion-mnist-dataset/fashion-mnist_train.csv', 'r'))
for line in reader:
     dataset_train.append(line)
        
# Read test dataset
dataset_test = []
reader = csv.DictReader(open('fashion-mnist-dataset/fashion-mnist_test.csv', 'r'))
for line in reader:
     dataset_test.append(line)
        
# Making a target dataset for each clothing option
# Finding out min and max values
min = 0
max = 0

for row in range(len(dataset_train)):
    if int(dataset_train[row]["label"]) < min:
        min = int(dataset_train[row]["label"])
        
    if int(dataset_train[row]["label"]) > max:
        max = int(dataset_train[row]["label"])
size = max - min + 1

dataset_train_target = np.zeros((size, len(dataset_train)))

for row in range(len(dataset_train)):
    x = int(dataset_train[row]["label"])
    dataset_train_target[x][row] = 1
    
   




In [19]:
def init_thetas(data, size):
    if len(data) == 0:
        return []
    thetas = np.zeros((size, len(data[0])))
    return thetas

def sigmoid(val):
    return 1 / (1 + math.exp(-val))

def calculate_hfunction(features, thetas):
    z = 0
    z += thetas[0]
    for f in range(1, len(thetas)):
        z += thetas[f] * features[f-1]
    h = sigmoid(z)
    return h

def calculate_cost_function(thetas, data, target):
    m = len(data)
    s = 0
        
    for index in range(len(data)):
        h = 0
        h += thetas[0]
        for k in range(1, len(thetas)):
            h += thetas[k] * data[index][k-1]
        h = sigmoid(h)
        s += (h - float(target[index]))*(h - float(target[index]))
    return (1/(2*m)) * (s)

def get_predictions(data, thetas):
    res = []
    for row in range(len(data)):
        z = np.matmul([data[row]],np.column_stack([thetas]))
        h = sigmoid(z)
        res.append(h[0,0])
    return res


In [13]:
#####################################
#                                   #
#    GRADIENT DESCENT ALGORITHM     #
#                                   #
#####################################


def gradient_descent(data, target, thetas, batch_size = 1, max_iterations = 100, stopCondition = 1e-04, learningRate = 1e-03, j_step=1000):
    done       = False
    m          = len(data)
    iterations = 0
    
    # After j_step iterations, compute cost function
    costs       = []
    itr_numbers = []
    
    retryCount = 0
    retryMax = 1000

    #startTime = current_time()
    
    while(iterations < max_iterations and not done):
        # Step through the dataset in chuncks
        for row in range(0, len(data), batch_size):
            new_thetas = thetas.copy()
            
        # Update theta 0 
            s = 0
            for offset in range(batch_size):
                if row + offset >= m:
                    break
                h = calculate_hfunction(data[row+offset], thetas)
                s = (h - float(target[row+offset]))
                
            new_thetas[0] = thetas[0] - ((learningRate / batch_size) * s)    
    
            # For each theta we do the following
            for k in range(1, len(thetas)):

                s = 0
                # We add every row of the dataset to the error calculation (Batch)
                for offset in range(batch_size):
                    if row + offset >= m:
                        break

                    h = calculate_hfunction(data[row+offset], thetas)
                    s += (h - float(target[row+offset])) * data[row+offset][k-1]

                # Updating the new thetas vector values
                new_thetas[k] = thetas[k] - ((learningRate / batch_size) * s)
            
            # keep a new cost value
            if iterations % j_step == 0:
                cost = calculate_cost_function(thetas, data, target)
                if len(costs)>0 and cost > costs[-1]:
                    learningRate /= 1.001
                    if retryCount < retryMax:
                        retryCount += 1
                    else:
                        iterations = max_iterations
                else:
                    retryCount = 0
                costs.append(cost)
                itr_numbers.append(iterations)
                
            iterations = iterations + 1
            if iterations >= max_iterations:
                break
    
    
             # If the change in value for new thetas is too small, we can stop iterating
            done = True
            for k in range(len(thetas)):
                done = abs(thetas[k] - new_thetas[k]) < stopCondition and done
            if done:
                break

            # Atualization of the values of the thetas
            thetas = new_thetas.copy()
    
    if iterations >= max_iterations:
        print("Stopped by number of iterations\n")
    if done:
        print("Stopped by convergence\n")
        
    #endTime = current_time()
    #print("RunTime = ", (endTime - startTime)/1000, " seconds")
    
    return thetas, itr_numbers, costs
    
    

In [35]:
#####################################
#                                   #
#     BATCH GRADIENT ALGORITHM      #
#                                   #
#####################################

thetas = init_thetas(dataset_train, size)
thetas_batch = init_thetas(dataset_train, size)


print(thetas.shape)

for model in range (size):
    target_train = dataset_train_target[model]
    max_iter = 100
    
    thetas_batch[model], itr_numbers_batch, costs_batch = gradient_descent(dataset_train, target_train, thetas[model],  batch_size=len(dataset_train))

(10, 786)


TypeError: 'list' object is not callable